In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../") # go to parent dir
from utils.data_simulator import DataSimulator
import numpy as np
import pandas as pd
import random
from utils.utils_syn import get_f1, summary
from sklearn.metrics import roc_auc_score
import multiprocessing

# for reproducibility
seed = 0
np.random.seed(seed)
random.seed(seed)

# number of valid IVs
p = 10

# are there dependencies within valid IVs?
cond_indp = False

# max degree of dependencies
d = 5

# number of invalid IVs
q = 8

# number of samples
n = 80000

# create a DataSimulator instance
mdl = DataSimulator(p,cond_indp=cond_indp,d=d,q=q,seed=seed)

# sample synthetic data from mdl
z, IVs, x, y = mdl.get_samples(n=n)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: 'DataFrame' object has no attribute 'append'

In [16]:
# Example usage
seed = 0
np.random.seed(seed)
random.seed(seed)

# Number of valid IVs
p = 10

# Are there dependencies within valid IVs?
cond_indp = False

# Max degree of dependencies
d = 5

# Number of invalid IVs
q = 8

# Number of samples
n = 80000

# Create a DataSimulator instance
mdl = DataSimulator(p, cond_indp=cond_indp, d=d, q=q, seed=seed, causal_effect=True, valid_z=True)

# Sample synthetic data from mdl
z, IVs, x, y = mdl.get_samples(n=n)

# Create DataFrame for analysis
data = pd.DataFrame({
    'z': z,
    'x': x,
    't': x,  # 't' should be sampled from BayesNetDataSimulator, correct this if necessary
    'y': y
})

# Display first few rows
print(data.head())

# Plotting the distributions of generated data
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
sns.countplot(x='z', data=data)
plt.title('Distribution of Z')

plt.subplot(2, 2, 2)
sns.countplot(x='x', data=data)
plt.title('Distribution of X')

plt.subplot(2, 2, 3)
sns.countplot(x='t', data=data)
plt.title('Distribution of T')

plt.subplot(2, 2, 4)
sns.countplot(x='y', data=data)
plt.title('Distribution of Y')

plt.tight_layout()
plt.show()

# Scatter plot to show relationships between variables
sns.pairplot(data, kind="scatter", diag_kind="kde")
plt.show()

AttributeError: 'DataFrame' object has no attribute 'append'